<a href="https://colab.research.google.com/github/Rudr16a/Twitter-Sentiment-Analysis/blob/main/Twitter_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install kaggle

Uploading Kaggle JSON file


In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


Imorting Twitter Sentiment dataSet through API

In [ ]:
!kaggle datasets download -d kazanova/sentiment140

Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other
 98% 79.0M/80.9M [00:02<00:00, 35.5MB/s]
100% 80.9M/80.9M [00:02<00:00, 33.9MB/s]


In [ ]:
#Extracting the compressed data set
from zipfile import ZipFile
data_set='/content/sentiment140.zip'

with ZipFile(data_set,'r') as zip:
  zip.extractall()
  print("The dataset is extracted")

The dataset is extracted


Importing the dependencies

In [ ]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
#Printing the stopwords in English
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

Data Processing

In [ ]:
Twitter_data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv', encoding='ISO-8859-1')

In [ ]:
#checking the no of rows and columns
Twitter_data.shape

(1599999, 6)

In [ ]:
#print first 5 rows of the data
Twitter_data.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [ ]:
#naming the columns and reading the data-set again

column_names = ['target', 'id', 'date', 'flag', 'user', 'text']
Twitter_data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv', encoding='ISO-8859-1', names=column_names)

In [ ]:
Twitter_data.shape

(1600000, 6)

In [ ]:
Twitter_data.head()

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [ ]:
#Counting the missing value in the Data-set
Twitter_data.isnull().sum()

target    0
id        0
date      0
flag      0
user      0
text      0
dtype: int64

In [ ]:
#checking the distribution of target columns
Twitter_data['target'].value_counts()

target
0    800000
4    800000
Name: count, dtype: int64

Convert the target "4" to "1"

In [ ]:
Twitter_data.replace({'target': {4: 1}}, inplace=True)

Twitter_data['target'].value_counts()

target
0    800000
1    800000
Name: count, dtype: int64

0 ---> negative tweet
1 ---> Positive tweet

Stemming

Stemming is process of reducing a word into its key word

In [ ]:
port_stem= PorterStemmer()

In [ ]:
def stemming(content):
  stemmed_content = re.sub('[^a-zA-Z]',' ',content)
  stemmed_content = stemmed_content.lower()
  stemmed_content = stemmed_content.split()
  stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content = ' '.join(stemmed_content)
  return stemmed_content

In [ ]:
Twitter_data['stemmed_content']= Twitter_data['text'].apply(stemming)

In [29]:
Twitter_data.head()

,target,id,date,flag,user,text,stemmed_content
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoot http twitpic com zl awww bummer sho...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,upset updat facebook text might cri result sch...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,kenichan dive mani time ball manag save rest g...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,whole bodi feel itchi like fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclass behav mad see


In [30]:
print(Twitter_data['stemmed_content'])

0          switchfoot http twitpic com zl awww bummer sho...
1          upset updat facebook text might cri result sch...
2          kenichan dive mani time ball manag save rest g...
3                            whole bodi feel itchi like fire
4                              nationwideclass behav mad see
                                 ...                        
1599995                           woke school best feel ever
1599996    thewdb com cool hear old walt interview http b...
1599997                         readi mojo makeov ask detail
1599998    happi th birthday boo alll time tupac amaru sh...
1599999    happi charitytuesday thenspcc sparkschar speak...
Name: stemmed_content, Length: 1600000, dtype: object


In [31]:
print(Twitter_data['target'])

0          0
1          0
2          0
3          0
4          0
          ..
1599995    1
1599996    1
1599997    1
1599998    1
1599999    1
Name: target, Length: 1600000, dtype: int64


In [32]:
#separating the data and label
X= Twitter_data['stemmed_content'].values
Y= Twitter_data['target'].values

In [33]:
print(X)

['switchfoot http twitpic com zl awww bummer shoulda got david carr third day'
 'upset updat facebook text might cri result school today also blah'
 'kenichan dive mani time ball manag save rest go bound' ...
 'readi mojo makeov ask detail'
 'happi th birthday boo alll time tupac amaru shakur'
 'happi charitytuesday thenspcc sparkschar speakinguph h']


In [34]:
print(Y)

[0 0 0 ... 1 1 1]


Splitting the data into test data and training data

In [35]:
X_train , X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.2,stratify= Y ,random_state=3)

In [37]:
print(X.shape,X_train.shape, X_test.shape)

(1600000,) (1280000,) (320000,)


In [38]:
print(X_train)

['arm fell comput chair go lean' 'chrishillfan got media next lucki'
 'suck havin watch yur mother sick cnt help' ...
 'melosm oooop sorri non exist miss call'
 'frowzl accord j rg kachelmann start rain tomorrow clock morn let hope right ja komm auch mit'
 'alcarlton lol idea mate go buy cloth much simpler option']


In [39]:
print(X_test)

['hate dog bark someth noth happen scari time tri sleep'
 'willi day either support cuz like guy' 'jordalynn yah pretti mess' ...
 'use spare usb flash drive virtual ram pc got gb memori'
 'sistatreen jealou miss beach'
 'sakura kurosaki prob hope good one lt ooh kind cake love cake']


Converting the textual data to numerical data

In [40]:
vectorizer= TfidfVectorizer()

X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [41]:
print(X_train)

  (0, 230023)	0.5302789468855067
  (0, 149604)	0.18999894424903027
  (0, 66262)	0.4703657150050708
  (0, 79295)	0.3591444105830402
  (0, 130345)	0.39805809775478695
  (0, 23107)	0.4171639689388593
  (1, 243427)	0.35968353780563106
  (1, 288268)	0.28409690701894913
  (1, 258288)	0.42799264451062413
  (1, 151539)	0.22265959091242618
  (1, 71762)	0.7464326512642685
  (2, 163634)	0.2761690912767651
  (2, 76994)	0.437205368032008
  (2, 366626)	0.2737474570064778
  (2, 274638)	0.3245662122941835
  (2, 457071)	0.49837923199930634
  (2, 436235)	0.22572360152384552
  (2, 160639)	0.4212655595518347
  (2, 386886)	0.2747790530029926
  (3, 172089)	0.26317971923417777
  (3, 384525)	0.33882627077985117
  (3, 256931)	0.4050476960176577
  (3, 438627)	0.26510430687972497
  (3, 365775)	0.3124620940507158
  (3, 412683)	0.3032716769864438
  :	:
  (1279998, 139300)	0.3779561489329452
  (1279998, 220549)	0.3779561489329452
  (1279998, 269095)	0.29047254123711924
  (1279998, 338984)	0.2926984744274037
  (1279

In [42]:
print(X_test)

  (0, 415689)	0.2518922757443892
  (0, 408499)	0.21003368771385605
  (0, 375101)	0.28488347461012686
  (0, 371029)	0.2464047959343191
  (0, 354711)	0.38027426988122187
  (0, 294474)	0.2984550266257167
  (0, 160330)	0.2625869593536074
  (0, 159203)	0.30050250593214706
  (0, 105377)	0.3245022701383146
  (0, 33329)	0.5023175290684292
  (1, 442056)	0.5541089394462928
  (1, 389414)	0.42351953294380973
  (1, 234202)	0.23372878235800623
  (1, 156037)	0.3169093785810737
  (1, 115722)	0.392787042356637
  (1, 93683)	0.21941299197386777
  (1, 87100)	0.39510766737101444
  (2, 452062)	0.4488951980065795
  (2, 322579)	0.30380202466576606
  (2, 261613)	0.3834893477036243
  (2, 201868)	0.7477521990696274
  (3, 456101)	0.9045922983924057
  (3, 267239)	0.42627781280421434
  (4, 435673)	0.17094573599581595
  (4, 368276)	0.24078151125805283
  :	:
  (319997, 432040)	0.3449292201083686
  (319997, 426743)	0.206243445111222
  (319997, 426714)	0.35562094758353474
  (319997, 377504)	0.34307286039499907
  (31999

Training the machine learning model ( Logistic Regression )

In [43]:
model = LogisticRegression(max_iter=1000)

In [44]:
model.fit(X_train,Y_train)

LogisticRegression(max_iter=1000)

Model Evaluation

In [45]:
#accuracy score on the training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction,Y_train)

In [47]:
print("Accuracy Score on the training data:", training_data_accuracy)


Accuracy Score on the training data: 0.81000390625


In [50]:
#accuracy score on the test data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction,Y_test)

In [51]:
print("Accuracy Score on the test data:", test_data_accuracy)

Accuracy Score on the test data: 0.7785375


Model accuracy = 77.8%

In [52]:
import pickle

In [53]:
filename = 'trained_model.sav'
pickle.dump(model, open(filename, 'wb'))

Using the saved model for future predictions

In [54]:
#loading the save model
loaded_model = pickle.load(open('/content/trained_model.sav', 'rb'))

In [58]:
X_new = X_test[200]

print(Y_test[200])

prediction = loaded_model.predict(X_new)
print(prediction)

if (prediction[0]==0):
  print('Negative tweet')
elif (prediction[0]==1):
  print('Positive tweet')

0
[0]
Negative tweet


In [61]:
X_new = X_test[13]

print(Y_test[13])

prediction = loaded_model.predict(X_new)
print(prediction)

if (prediction[0]==0):
  print('Negative tweet')
elif (prediction[0]==1):
  print('Positive tweet')

1
[1]
Positive tweet
